In [19]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [20]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [21]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [24]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [67]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [59]:
def getStakeAndRewards():
    stake = dict()
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            effective_stake = int(float(i['metrics']['total-effective-stake']))
            stake[address] = effective_stake
            reward[address] = reward_accumulated
    return stake, reward

In [60]:
def getRewards():
    reward = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            reward[address] = reward_accumulated
    return reward

In [99]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            map3[k] = v - map1[k]
    return map3

In [100]:
def mapFilter(map1, map2):
    map3 = dict()
    for k, v in map1.items():
        if k in map2:
            map3[k] = v
    return map3

In [107]:
def identical(l1, l2):
    return l1 == l2

In [102]:
def extract(lst): 
    return [item[0] for item in lst] 

In [111]:
def R3_test():
    print("Test-R3: High stakers earn more reward than low stakers")
    num = int(input("enter the number of blocks you want to test: "))
    
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    while block < last_block:
        block = getBlockNumber()
        
    new_epoch_block = block + 1
    while block < new_epoch_block:
        block = getBlockNumber()
    
    print("new epoch first block reached", new_epoch_block, "will begin testing")
    new_epoch_block = block + 1
    
    stakes, rewards = getStakeAndRewards()
    sorted_stakes = sorted(stakes.items(), key=lambda kv: kv[1], reverse = True)
    sorted_rewards = sorted(rewards.items(), key=lambda kv: kv[1], reverse = True)
    
    print("obtained first block stakes and rewards", "total stakes found=", \
          len(stakes), "total rewards found=", len(rewards))
    
    while block < new_epoch_block:
        block = getBlockNumber()
        
    print("new epoch second block reached", block, "will begin comparing stakes and rewards")
        
    new_rewards = getRewards()
    
    last_block = block
    current_block = last_block

    iterations = 0
    while iterations < num:
        reward_diffs = diffAndFilter(rewards, new_rewards)
        latest_stakes = mapFilter(stakes, new_rewards)
        sorted_stakes = sorted(latest_stakes.items(), key=lambda kv: kv[1], reverse = True)
        sorted_reward_diffs = sorted(reward_diffs.items(), key=lambda kv: kv[1], reverse = True)
        stake_keys = extract(sorted_stakes)
        reward_keys = extract(sorted_reward_diffs)
        print("comparison to begin, two lengths:", len(latest_stakes), len(sorted_rewards))
        if identical(stake_keys, reward_keys) == False:
            print("Test-R3: Fail")
            print("validators sorted by stakes: ", stake_keys)
            print("validators sorted by reward: ", reward_keys)
        else:
            print("Test-R3: Success")
            
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        new_rewards = getRewards()
        iterations = iterations + 1

In [112]:
R3_test()

Test-R3: High stakers earn more reward than low stakers
enter the number of blocks you want to test: 1
current and last block numbers 32061 32071
new epoch first block reached 32072 will begin testing
obtained first block stakes and rewards total stakes found= 60 total rewards found= 60
new epoch second block reached 32073 will begin comparing stakes and rewards
comparison to begin, two lengths: 60 60
validators sorted by stakes:  ['one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9', 'one1s7fp0jrmd97estwye3mhkp7xsqf42vn5x2sfqy', 'one1u6c4wer2dkm767hmjeehnwu6tqqur62gx9vqsd', 'one1ha4zg6xstkpmvvg87eghtg93u0exvzs3kl0ew0', 'one1vdvxewvahkm2pn4jmvwvgsfqct53s86r0y505l', 'one1gjsxmewzws9mt3fn65jmdhr3e4hel9xza8wd6t', 'one16226n7h7fu326plzlqt4k60yfyza5vk299ypr2', 'one1uvt0yl7dxyt9rh37gzm49vy3pgf0c3aud4k5p5', 'one1yym297x9elfeqj0ru8pk3qcnhm60e0sgxgc7zd', 'one103q7qe5t2505lypvltkqtddaef5tzfxwsse4z7', 'one1marnnvc8hywmfxhrc8mtpjkvvdt32x9kxtwkvv', 'one1egemh5e9xjy3x8d3cq0kq7mw4sw4jjwgkc7axs', 'one17tj2jje